In [1]:
import cv2
import mediapipe as mp
import time  # Import the time module

# MediaPipe Pose 모델 로드
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 카운터 초기화
count = 0
is_up = False  # 팔굽혀펴기가 올라가고 있는지 여부
initial_detection = False  # 초기 검출 플래그
start_displayed = False  # 시작 메시지 표시 여부
count_initialized = False  # 초기화 플래그
preparation_ready_time = None  # 시간 기록 변수

# 이전 프레임에서 판단한 자세
prev_pose = None

# 이전에 일직선에 있던 시간
straight_line_time = 0
straight_line_duration = 2.0  # 2 seconds

# 마지막 팔굽혀펴기 감지 시간
last_pushup_time = time.time()
print("감지시간:",last_pushup_time)

# 올바른 팔굽혀펴기 포즈를 판단하는 함수
def is_correct_pushup_pose(keypoints):
    global count, is_up, prev_pose, straight_line_time, preparation_ready_time, last_pushup_time

    # 여기서는 예시로 왼쪽 어깨, 왼쪽 팔꿈치, 왼쪽 손목, 오른쪽 어깨, 오른쪽 팔꿈치, 오른쪽 손목을 활용합니다.
    left_shoulder = keypoints[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    left_elbow = keypoints[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    left_wrist = keypoints[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_shoulder = keypoints[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    right_elbow = keypoints[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    right_wrist = keypoints[mp_pose.PoseLandmark.RIGHT_WRIST.value]

    # 팔굽혀펴기 시작자세 판단 (손목, 팔꿈치, 어깨가 일직선)
    if (
        left_shoulder.y < left_elbow.y < left_wrist.y
        and right_shoulder.y < right_elbow.y < right_wrist.y
    ):
        print("업데이트전 시간:",last_pushup_time)
        current_pose = "up"
        straight_line_time = time.time() if straight_line_time == 0 else straight_line_time
        last_pushup_time = time.time()  # 감지된 팔굽혀펴기 시간 업데이트
        print("업데이트 시간:",last_pushup_time)
    else:
        current_pose = "down"
        straight_line_time = 0

    # 판단한 자세가 이전과 다를 때 count 증가
    if current_pose != prev_pose:
        if current_pose == "up":
            # Check if Preparation Pose Ready message was displayed
            if preparation_ready_time is not None:
                count += 1
                if time.time() - last_pushup_time >= 2.0 and not is_up:
                   print("이떄 end표시:",time.time()-last_pushup_time)
                   cv2.putText(frame, "End!", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                is_up = True
                print("팔굽혀펴기 올라감! Count:", count)
                print("이제 내려가세요.")
            else:
                print("Preparation Pose Ready! 메시지가 표시되기를 기다리는 중입니다.")
        else:
            is_up = False
            print("이제 정자세로 돌아오세요.")

    # 팔굽혀펴기 준비자세 판단
    if straight_line_time > 0 and time.time() - straight_line_time >= straight_line_duration:
        if preparation_ready_time is None:  # Set the preparation_ready_time only once
            cv2.putText(frame, "Preparation Pose Ready!", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            preparation_ready_time = time.time()  # 기록해둠
            time.sleep(3)

    prev_pose = current_pose

    return True if current_pose == "up" else False

# 동영상 파일 경로
video_path = 'C://Users//dokgo//Desktop//final//qqq.mp4'

# 웹캠으로부터 비디오 스트림 읽기
cap = cv2.VideoCapture(video_path)

# 비디오 프레임 크기 조정
target_width, target_height = 1024, 768

while cap.isOpened():
    # 초기 검출 플래그 초기화
    
    # 프레임 읽기
    ret, frame = cap.read()
    if not ret:
        break
    # End! 표시

      #  if time.time() - last_pushup_time >= 2.0 and not is_up:
      #      print(time.time()-last_pushup_time)
      #      cv2.putText(frame, "End!", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    frame = cv2.resize(frame, (target_width, target_height))

    # Mediapipe Pose 적용
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # 감지된 포즈 점 찾기
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        # 여기에서 is_correct_pushup_pose 함수를 사용하여 올바른 팔굽혀펴기 포즈인지 확인
        is_correct_pushup_pose(landmarks)

        # 키포인트 시각화 및 선 그리기
        for lm_id, landmark in enumerate(landmarks):
            h, w, _ = frame.shape
            cx, cy = int(landmark.x * w), int(landmark.y * h)

            # 팔 굽히기에 필요한 포인트만 시각화
            if lm_id in [mp_pose.PoseLandmark.LEFT_SHOULDER.value, mp_pose.PoseLandmark.LEFT_ELBOW.value, mp_pose.PoseLandmark.LEFT_WRIST.value,
                          mp_pose.PoseLandmark.RIGHT_SHOULDER.value, mp_pose.PoseLandmark.RIGHT_ELBOW.value, mp_pose.PoseLandmark.RIGHT_WRIST.value]:
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

        # 정의된 포즈 점을 사용하여 선 그리기
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

        cv2.line(frame, (int(left_shoulder.x * w), int(left_shoulder.y * h)),
                 (int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * w), int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * h)),
                 (255, 0, 0), 2)

        cv2.line(frame, (int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * w), int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * h)),
                 (int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * w), int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * h)),
                 (255, 0, 0), 2)

        cv2.line(frame, (int(right_shoulder.x * w), int(right_shoulder.y * h)),
                 (int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * w), int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * h)),
                 (255, 0, 0), 2)

        cv2.line(frame, (int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * w), int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * h)),
                 (int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * w), int(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * h)),
                 (255, 0, 0), 2)

    # 계속해서 count 표시
    if count_initialized:  # Display count only after initialization
        cv2.putText(frame, f"Count: {count}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Up 또는 Down 메시지 표시
        if is_up:
            cv2.putText(frame, "More Down!", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame, "Up!", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 팔굽혀펴기 준비자세가 될 때까지 메시지 유지
    if preparation_ready_time is None:
        cv2.putText(frame, "Preparating Pose", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)


    # 화면에 출력
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame.flags.writeable = True
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("LWBM", frame)

    # 초기화면에서 count 초기화
    if not count_initialized and preparation_ready_time is not None:
        count = 0
        count_initialized = True

    # 종료 조건
    if cv2.waitKey(1) & 0xFF == 27:  # ESC 키를 누르면 종료
        break

# 종료
cap.release()
cv2.destroyAllWindows()


In [3]:
# 동영상 파일 경로
video_path = 'C://Users//dokgo//Desktop//final//qqq.mp4'

# 웹캠으로부터 비디오 스트림 읽기
cap = cv2.VideoCapture(video_path)